Import all of the required packages for this notebook

In [1]:
import sys 
import os
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx
import pandas as pd
import plotly.plotly as py
import plotly.graph_objs as go
import plotly.figure_factory as ff
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
from ipywidgets import widgets, interact, interactive
init_notebook_mode(connected=True)

#sys.path.append(os.path.abspath("/Users/mk100/Documents/Code/TITAN/src"))
sys.path.append(os.path.relpath("../src"))

import ABM_core as tc
import MP_simulation as tt
import params
import numpy as np

In [2]:
out = widgets.Output(layout={'border': '1px solid black'})
out_network = widgets.Output(layout={'border': '1px solid black'})
out_distrib = widgets.Output(layout={'border': '1px solid black'})
out_df = widgets.Output(layout={'border': '1px solid black'})
df = pd.DataFrame()
RSLTS = {}

In [3]:
def setParams(burnDur, calSA, calSP, calNA, calNP, calMort, calRRDx, calRRArt,calPXmn,calIncar,calARTcov):
    params.burnDuration=burnDur
    params.cal_SexualActScaling=calSA
    params.cal_SexualPartScaling=calSP
    params.cal_NeedleActScaling=calNA
    params.cal_NeedlePartScaling=calNP
    params.cal_Mortality=calMort
    params.cal_RR_Dx=calRRDx
    params.cal_RR_HAART=calRRArt
    params.cal_pXmissionScaling=calPXmn
    params.cal_IncarP=calIncar
    params.cal_ART_cov=calARTcov

y=interactive(setParams,
              burnDur=widgets.IntSlider(description='Burn',
                                       continuous_update=False, 
                                       min=0, 
                                       value=params.burnDuration),
              calSA=widgets.FloatSlider(description='Sex Acts',
                                       continuous_update=False, 
                                       min=0., 
                                       max=3.0,
                                       value=params.cal_SexualActScaling),
              calSP=widgets.FloatSlider(description='Sex Ptns',
                                       continuous_update=False, 
                                       min=0., 
                                       max=3.0,
                                       value=params.cal_SexualPartScaling),
              calNA=widgets.FloatSlider(description='Needle Acts',
                                       continuous_update=False, 
                                       min=0., 
                                       max=3.0,
                                       value=params.cal_NeedleActScaling),
              calNP=widgets.FloatSlider(description='Needle Ptns',
                                       continuous_update=False, 
                                       min=0., 
                                       max=3.0,
                                       value=params.cal_NeedlePartScaling),
              calMort=widgets.FloatSlider(description='Mortality',
                                       continuous_update=False, 
                                       min=0., 
                                       max=3.0,
                                       value=params.cal_Mortality),
              calRRDx=widgets.FloatSlider(description='RR Dx',
                                       continuous_update=False, 
                                       min=0., 
                                       max=3.0,
                                       value=params.cal_RR_Dx),
              calRRArt=widgets.FloatSlider(description='RR ART',
                                       continuous_update=False, 
                                       min=0., 
                                       max=3.0,
                                       value=params.cal_RR_HAART),
              calPXmn=widgets.FloatSlider(description='P Xmission',
                                       continuous_update=False, 
                                       min=0.,
                                       max=3.0,
                                       value=params.cal_pXmissionScaling),
              calIncar=widgets.FloatSlider(description='P Incar',
                                       continuous_update=False, 
                                       min=0., 
                                       max=3.0,
                                       value=params.cal_IncarP),
              calARTcov=widgets.FloatSlider(description='ART Cov',
                                       continuous_update=False, 
                                       min=0.,
                                       max=3.0,
                                       value=params.cal_ART_cov),
             )

In [4]:
nMC = widgets.IntSlider(description='nMC',continuous_update=False, min=0, max=100, value=1)
nPop = widgets.IntSlider(description='nPop',continuous_update=False, min=0, max=100000, value=100)
tMax = widgets.IntSlider(description='tMax',continuous_update=False, min=0, max=120, value=12)
seed = widgets.IntSlider(description='seed',continuous_update=False, min=-1,max=10, value=1)
draw = widgets.Checkbox(description='Draw Network',value=True,disabled=False)
uniqueseed = widgets.Checkbox(description='Unique Seed',value=False,disabled=False)
ui_params = widgets.VBox([nMC, nPop, tMax, seed, draw,uniqueseed])



prog = widgets.IntProgress(
        value=0,
        min=0,
        max=tMax.value,
        step=1,
        description='Progress:',
        bar_style='info'
        )
run = widgets.Button(
    description='Run Model',
    disabled=False,
    button_style='success', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Start model with given params',
    icon='play'
)
reset = widgets.Button(
    description='Reset',
    disabled=False,
    button_style='danger', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Clear outputs',
    icon='stop'
)

ui_buttons = widgets.HBox([run, reset])
ui = widgets.VBox([ui_params, ui_buttons, prog])

tab_nest = widgets.Tab()
tab_nest.children = [ui,y]
tab_nest.set_title(0, 'Main')
tab_nest.set_title(1, 'Advanced Params')

In [15]:
def on_run_clicked(b):
    reset_outputs()
    #tt.open_inputs()
    #global SIM
    global df
    with out:
        if nMC.value <= 1:
            SIM = makeModel(nMC.value, nPop.value, tMax.value, seed.value)
            SIM.run()
            #SIM = tt.simulation(nMC.value, 1, tMax.value, nPop.value, "", {}, seed.value)
            RSLTS = SIM.return_results()
            print RSLTS
            #df.append(pd.DataFrame(RSLTS))
        else:
            RSLTS = tt.simulation(nMC.value, 1, tMax.value, nPop.value, "", {}, seed.value, uniqueSeed=uniqueseed.value)
            #df.append(pd.DataFrame(RSLTS))
            print RSLTS
        df = df.append(pd.DataFrame(RSLTS))
        with out_distrib:
            if draw.value:
                for result_property in sorted(RSLTS):
                    #print result_property, rslts[result_property]
                    lists = sorted(RSLTS[result_property].items())
                    if lists:
                        x,y = zip(*lists)
                        plt.title("%s vs Time\nTime"%(result_property))
                        plt.xlabel("Time")
                        plt.plot(x,y)
                        plt.xticks(np.arange(0, tMax.value+1, step=12))
                        plt.show()

                #draw_Histogram()
        with out_network:
            if draw.value:
                #TMP.networkGraph.visualize_network(coloring='HIV',node_size=25)
                #TMP.networkGraph.draw_histogram(t)
                pass

        #df = df.append(pd.DataFrame(RSLTS))
        with out_df:
            #table = ff.create_table(df)
            #iplot(table)
            #display(df)
            df.head()

            data = []

            for col in df.columns:
                data.append(  go.Box( y=np.mean(df[col]), name=col, showlegend=False ) )
            #data.append( go.Scatter( x = df.columns, y = df.mean(), mode='lines', name='mean' ) )
            iplot(data)
            #plt.figure()
            #bp = df.boxplot()

def on_reset_clicked(b):
    global df
    df=pd.DataFrame()
    reset_outputs()
    tmp = seed.value
    seed.value=0
    seed.value=tmp
    prog.value=0

def reset_outputs():
    outs = [out, out_network, out_distrib, out_df]
    for o in outs:
        o.clear_output()

def makeModel(MC, N, T, S):
    #global SIM
    #global G
    #TMP = tc.HIVModel(N, T, {}, S)
#     if sysprint.value:
#         tt.enablePrint()
#     else:
#         tt.blockPrint()
    SIM = tt.HIVModel(N=nPop.value, tmax=tMax.value, parameter_dict={}, rseed=seed.value, 
                      runtime_diffseed=uniqueseed.value, model='ASDF', network_type='ASDF')
    #TMP.TimeStep = 0
    #G = TMP.networkGraph.G
    return SIM


run.on_click(on_run_clicked)
reset.on_click(on_reset_clicked)
widgets.jslink((tMax, 'value'), (prog, 'max'))
init = widgets.interactive_output(makeModel, {'MC': nMC, 'N': nPop, 'T': tMax, 'S': seed})
main = widgets.HBox([tab_nest, init])

In [6]:
display(main)

HBox(children=(Tab(children=(VBox(children=(VBox(children=(IntSlider(value=1, continuous_update=False, description=u'nMC'), IntSlider(value=100, continuous_update=False, description=u'nPop', max=100000), IntSlider(value=12, continuous_update=False, description=u'tMax', max=120), IntSlider(value=1, continuous_update=False, description=u'seed', max=10, min=-1), Checkbox(value=True, description=u'Draw Network'), Checkbox(value=False, description=u'Unique Seed'))), HBox(children=(Button(button_style=u'success', description=u'Run Model', icon=u'play', style=ButtonStyle(), tooltip=u'Start model with given params'), Button(button_style=u'danger', description=u'Reset', icon=u'stop', style=ButtonStyle(), tooltip=u'Clear outputs'))), IntProgress(value=0, bar_style=u'info', description=u'Progress:', max=12))), interactive(children=(IntSlider(value=0, continuous_update=False, description=u'Burn'), FloatSlider(value=1.0, continuous_update=False, description=u'Sex Acts', max=3.0), FloatSlider(value=1.0, continuous_update=False, description=u'Sex Ptns', max=3.0), FloatSlider(value=2.0, continuous_update=False, description=u'Needle Acts', max=3.0), FloatSlider(value=1.0, continuous_update=False, description=u'Needle Ptns', max=3.0), FloatSlider(value=0.5, continuous_update=False, description=u'Mortality', max=3.0), FloatSlider(value=0.53, continuous_update=False, description=u'RR Dx', max=3.0), FloatSlider(value=1.0, continuous_update=False, description=u'RR ART', max=3.0), FloatSlider(value=1.0, continuous_update=False, description=u'P Xmission', max=3.0), FloatSlider(value=1.0, continuous_update=False, description=u'P Incar', max=3.0), FloatSlider(value=1.0, continuous_update=False, description=u'ART Cov', max=3.0), Output()), _dom_classes=('widget-interact',))), _titles={u'1': 'Advanced Params', u'0': 'Main'}), Output()))

In [7]:
out

Output(layout=Layout(border=u'1px solid black'))

In [8]:
out_distrib

Output(layout=Layout(border=u'1px solid black'))

In [9]:
out_df

Output(layout=Layout(border=u'1px solid black'))

In [21]:
#print(df)
#df.head()

data = []

for col in df.columns:
    data.append(  go.Box( y=df[col], name=col, showlegend=False ) )
#data.append( go.Scatter( x = df.columns, y = df.mean(), mode='lines', name='mean' ) )
iplot(data)


def draw_Histogram():
    data = [go.Bar(
                y=(nx.degree_histogram(G)),
        marker=dict(
            color='rgb(49,130,189)'
        )
        )]
    layout = go.Layout(
        title='Partner Degree Distribution',
        autosize=False,
        width=400,
        height=300,
        margin=go.Margin(
        l=50,
        r=50,
        b=50,
        t=50,
        pad=4
    ),
        xaxis=dict(
            range=[-1,10],
            tickfont=dict(
                size=14,
                color='rgb(107, 107, 107)'
            )
        ),
        yaxis=dict(
            title='Number of Agents (N)',
            titlefont=dict(
                size=16,
                color='rgb(107, 107, 107)'
            ),
            tickfont=dict(
                size=14,
                color='rgb(107, 107, 107)'
            )
        ),
        legend=dict(
            x=0,
            y=1.0,
            bgcolor='rgba(255, 255, 255, 0)',
            bordercolor='rgba(255, 255, 255, 0)'
        ),
        barmode='group',
        bargap=0.15,
        bargroupgap=0.1
    )
    fig = go.Figure(data=data, layout=layout)
    iplot(fig)


nx.degree_mixing_dict(G)

nx.number_connected_components(G)
nx.is_attracting_component(G)

for a in nx.connected_components(G):
    print len(a)
    for p in a:
        print p

df = pd.read_csv("https://raw.githubusercontent.com/plotly/datasets/master/school_earnings.csv")

table = ff.create_table(df)
iplot(table, filename='jupyter-table1')